# `Python to Postgresql (Psycopg2)`: 
# `Create/Use (VIEWS)` | `Bonus: Date-Time Queries`

**<font color=red>Mr Fugu Data Science</font>**


# (◕‿◕✿)

[Github](https://github.com/MrFuguDataScience) | [Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw?view_as=subscriber)

`------------------`

`Purpose & Outcome`:

+ Describe pursose and uses of `views`
+ Create 2 View examples
   + Query Date Information

+ If you are working in a real world scenario, you will most likely need to hide your credentials,   permissions, login & password
that is where these files shine. 

[create init and config files](https://towardsdatascience.com/python-and-postgresql-how-to-access-a-postgresql-database-like-a-data-scientist-b5a9c5a0ea43) | [Python_configparser doc](https://docs.python.org/3/library/configparser.html) | [Postgres_tutorial_Python_PSQL](https://www.postgresqltutorial.com/postgresql-python/connect/)

1<sup>st</sup>) : create the initialization file:

+ use your terminal or notepad, unless your favorite editor directly converts to this format `.ini` file.
    + on `Mac` I did `touch database_file_init.ini`
        + `vi database_file.ini`     *then inside the file type following lines:*
            + `[postgresql]`
            + `host=localhost`
            + `database=what_databse_you_want_to_access`
            + `user= some_user_you_created_for_this_user_in_psql`
            + `password= some_password_you_have_to_this_db`
            
`------------------------------------------------`

2<sup>nd</sup>) : now the `config.py` file this will be used to take data from `init file` and outputs a `dictionary`.

+ this file will look like this as an **example**:
`{‘host’: ‘localhost’, ‘database’: ‘suppliers’, ‘user’: ‘postgres’, ‘password’: ‘postgres’}` when it is read.

`HERE IS THE CODE`:

`____________________________________________________`

#!/usr/bin/python    (can also do virtenv,env)

from configparser import ConfigParser
 
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    
    # Checks to see if section (postgresql) parser exists
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
         
    # Returns an error if a parameter is called that is not listed in the initialization file
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db
    
    
**This code was adpated from online material on Postgres website and used in 1<sup>st</sup> & 3<sup>rd</sup> link above**


`---------------------------------------`

# Install Psycopg2:

+ `python -m pip install psycopg2`

* if this doesn't work, try changing `pip` to `pip3` depending on what version of Python you are using

+ if that doesn't work either try to do: `conda install -c anaconda psycopg2`


**For my installation I found problems with installation: I did two steps**: `pip3 install psycopg2` followed by the `conda install -c anaconda psycopg2`. The first install showed complete but did not work, I feel that I may have been imcomplete with all the dependencies needed. Also, I was getting a `Python 2.7 error hashing`. `I suggest that you first check the version of Python and Location PATH. I feel that my default Location may be anaconda calling Python if I remember correctly.`

+ I thought this was from using a wrong default version of Python, changed my default and still didn't work. That was not the entire case, try the above and hope this works for you.

+ Separate source for installing: depending on situation. [install on Mac](https://www.youtube.com/watch?v=N4RxnQH2pVY)

`____________________________________________`

If you do not want to create init or config files: do something similar to this

**`import psycopg2`**

**`conn = psycopg2.connect("dbname=test, user=postgres")`** #Connect to an existing database

**`cur = conn.cursor( )`** #Open a cursor to perform database operations`


`--------------------------------------------`

In [2]:
import psycopg2             # python->psql connection
# 
import pandas as pd         # create dataframes 
# import calendar             # convert int to month
# import matplotlib.pyplot as plt  # barchar plot
import seaborn as sns

# Import the 'config' function from the config_user_dta.py file:
from config_user_dta import config

In [3]:
# Get the config params
params_ = config()

# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)

# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()

conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)

In [281]:
sql_="SELECT * FROM  ppl_cpu_purchases LIMIT 2"
cur.execute(sql_)
cur.fetchall()

# pd.DataFrame(cur.fetchall(),columns=['credit_card','email','first_name','last_name',
# 'primary_phone','cpu','purchase_date'])

[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148',
  'Intel Core i1-7554K',
  datetime.date(2019, 10, 31)),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716',
  'AMD Ryzen 1 5827X',
  datetime.date(2017, 7, 16))]

# What is a `View`?

+ It can be thought of as a 'psuedo-table'. It is not a real table but instead, a database object that stores a query. 
    + More or less, think of a `View` as a representation of your data and does not store anything unless you are using a `materialzed view`. 
    + Materialized views: are very fast lookups and often used in data-warehousing. 

+ The view, can be used for subsetting data, joining multiple tables, even granting permissions are an option. 
    
+ **General Setup**:


`CREATE OR REPLACE view_name 
AS 
your_query`

OR

`CREATE [ OR REPLACE ] [ TEMP | TEMPORARY ] VIEW name [ ( column_name [, ...] ) ]
    [ WITH ( view_option_name [= view_option_value] [, ... ] ) ]
    AS query`


# When Would I Use a `View`?

+ Consider if you are forming complex queries, running specific queries often or you may need to use those results for some other work you are doing. Then a `View` is a great option. 

+ Why: Because, you can call the VIEW directly, because it has already stored your complex query and you can do further queries or processing if needed. 

`----------------------------------------------------`

# EX.1) Create View

+ Your Supervisor wants you to create a new table showing first,last name with credit card number and the cpu with purchase date. 
    + You have 2 tables you need to put together (via `inner join`)
+ You are accessing these data often and would benefit from creating a view.

In [11]:
sql_view=" CREATE VIEW customer_cpu_sales AS \
SELECT staging_fake_ppl.first_name || ' ' || staging_fake_ppl.last_name \
AS full_name, staging_fake_ppl.credit_card AS card,\
staging_fake_cpu_purchases.cpu,staging_fake_cpu_purchases.purchase_date \
 FROM staging_fake_cpu_purchases INNER JOIN staging_fake_ppl ON \
 staging_fake_ppl.credit_card=staging_fake_cpu_purchases.credit_card"

cur.execute(sql_view)


In [277]:
# QUERY OUR first VIEW:

s="SELECT * FROM customer_cpu_sales LIMIT 4"
cur.execute(s)
cur.fetchall()



[('Nasreen Angel',
  '1000-1293-3496-7245',
  'AMD Ryzen 4 6221X',
  datetime.date(2018, 5, 27)),
 ('Qirsten el-Azimi',
  '1000-7698-8868-7669',
  'Intel Core i5-2492K',
  datetime.date(2020, 3, 9)),
 ('Alejandro Batchelor',
  '1002-7633-3301-5737',
  'Intel Core i2-7948K',
  datetime.date(2017, 10, 16)),
 ('Gabriel Salazar',
  '1003-1072-2532-5536',
  'AMD Ryzen 5 5684X',
  datetime.date(2020, 5, 23))]

In [ ]:
# Update View: for example change the (name of our VIEW)

"ALTER VIEW customer_master RENAME TO customer_info"

In [ ]:
# Delete View:

"DROP VIEW thne_your_viewName"

# Query and print days since purchase

In [71]:
sales_byDate="select date_trunc('day',now() - purchase_date::date) from customer_cpu_sales"
cur.execute(w)
cur.fetchone()

(datetime.timedelta(days=240),)

# Ex. 2) `View 02` Storing dates since Purchase

+ Now, you are asked to store the days elapsed since purchase tracked from today

In [74]:

cust_warr_view="CREATE VIEW customer_warranty AS SELECT *,\
date_trunc('day',now() - purchase_date::date) \
AS 'days_since_purchase' FROM ppl_cpu_purchases"
cur.execute(cust_warr_view)
# cur.fetchall()



In [272]:
sqq="select * from customer_warranty limit 2"
cur.execute(sqq)
cur.fetchall()


[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148',
  'Intel Core i1-7554K',
  datetime.date(2019, 10, 31),
  datetime.timedelta(days=241)),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716',
  'AMD Ryzen 1 5827X',
  datetime.date(2017, 7, 16),
  datetime.timedelta(days=1078))]

# Date-Time Queries:

+ Use `Extract function` with `CASE WHEN`

* **Question**: Assume you have to track the age of your customer purchases for warranty information. `Imagine that you only have a warranty of (6 months: AKA 180 days)`.

In [283]:
xc="SELECT * FROM customer_warranty where extract(days from warranty) < 180 limit 3"

cur.execute(xc)
# cur.fetchall()

pd.DataFrame(cur.fetchall(),columns=['credit_card','email','first_name','last_name',
'primary_phone', 'cpu','purchase_date','days_since_purchase'])

,credit_card,email,first_name,last_name,primary_phone,cpu,purchase_date,days_since_purchase
0,7536-4552-2504-7339,ybt@ehlczprdt.lbd,Everett,Vigil,9722173416,Intel Core i3-2155K,2020-03-07,113 days
1,2487-1704-2828-6891,urszdy@iplejhf.dwl,Paige,Burton,2178936729,AMD Ryzen 4 2978X,2020-04-09,80 days
2,8464-3306-8207-9275,pvjderban@alfcbr.dax,Ann,Reish,5236872168,AMD Ryzen 4 5508X,2020-04-26,63 days


# Not All Queries Go AS PLANNED:

In [319]:
dates_n_warr="SELECT *,\
date_trunc('day',now() - purchase_date::date) \
AS days_since_purchase, \
CASE WHEN extract(day from purchase_date::date) < 180 \
THEN 'Yes' ELSE 'No' END AS warranty FROM ppl_cpu_purchases"

cur.execute(dates_n_warr)
cur.fetchall()

[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148',
  'Intel Core i1-7554K',
  datetime.date(2019, 10, 31),
  datetime.timedelta(days=241),
  'Yes'),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716',
  'AMD Ryzen 1 5827X',
  datetime.date(2017, 7, 16),
  datetime.timedelta(days=1078),
  'Yes'),
 ('4435-3866-1076-3595',
  'dvyco@tkzhsop.zxg',
  'Connor',
  'Powers',
  '3627413915',
  'Intel Core i5-9457K',
  datetime.date(2019, 3, 24),
  datetime.timedelta(days=462),
  'Yes'),
 ('3489-7099-9906-8660',
  'fy@uvfhplatmz.cam',
  'Kylie',
  'Her',
  '3562764561',
  'AMD Ryzen 4 3401X',
  datetime.date(2019, 6, 15),
  datetime.timedelta(days=379),
  'Yes'),
 ('8631-4500-5666-1510',
  'rztkvliou@dkeinhgysf.deo',
  'Anthony',
  'Vo',
  '7345795348',
  'Intel Core i6-7283K',
  datetime.date(2019, 11, 11),
  datetime.timedelta(days=230),
  'Yes'),
 ('1459-9918-1722-7369',
  'jofmezlbp@iw.evx',
  'Mutammam',
  'Mares',
 

In [264]:
# cur.execute("select * from ppl_cpu_purchases extract(day from purchase_date::date) < 180 limit 3")

cur.execute('select version()')
cur.fetchall()

[('PostgreSQL 12.2 on x86_64-apple-darwin17.7.0, compiled by Apple LLVM version 10.0.0 (clang-1000.11.45.5), 64-bit',)]

# Create Query:
+ **By Date Range (Month & Year) of CPU's Sold**

*Assume you had to find the monthly sales of cpu*

In [316]:

cpu_g="SELECT pg_catalog.date(date_trunc('month',purchase_date)) AS txn_month, \
count(cpu) as monthly_sum \
FROM ppl_cpu_purchases GROUP BY txn_month ORDER BY txn_month DESC"

cur.execute(cpu_g)
pd.DataFrame(cur.fetchall(),columns=['Month_yr','Items Sold']).head(7)

,Month_yr,Items Sold
0,2020-05-01,163
1,2020-04-01,152
2,2020-03-01,150
3,2020-02-01,157
4,2020-01-01,139
5,2019-12-01,185
6,2019-11-01,152


# THROW a <font color=red>Like</font>
# <font color=red>SUBscribe</font> & Turn ON Notification 🔔 Bell 

# Citations & Help:

https://www.tutorialspoint.com/postgresql/postgresql_views.htm

https://www.postgresqltutorial.com/managing-postgresql-views/

https://www.datacamp.com/community/tutorials/materialized-views-postgresql

https://www.tutorialspoint.com/postgresql/postgresql_date_time.htm

https://stackoverflow.com/questions/45487731/postgres-get-number-of-days-since-date

https://www.sqlservertutorial.net/sql-server-date-functions/sql-server-datediff-function/

https://www.plus2net.com/sql_tutorial/date-lastweek.php

https://www.postgresqltutorial.com/postgresql-case/

https://stackoverflow.com/questions/17492167/group-query-results-by-month-and-year-in-postgresql

https://stackoverflow.com/questions/6133107/extract-date-yyyy-mm-dd-from-a-timestamp-in-postgresql

https://w3resource.com/PostgreSQL/pl-pgsql-control-structures.php